In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import datetime
import requests
import json

In [3]:
# Read trips text files into pandas dataframe
leavetimes = pd.read_csv("../../Data/rt_leavetimes_2017 - Sample Data.txt", sep=';')
trips_16 = pd.read_csv("../../Data/rt_trips_2016_I_DB.txt", sep=';')
trips_17 = pd.read_csv("../../Data/rt_trips_2017_I_DB.txt", sep=';')

In [4]:
trips = pd.concat([trips_16, trips_17], join="inner")

In [5]:
leavetimes.head(1)

,datasource,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,vehicleid,passengers,passengersin,passengersout,distance,suppressed,justificationid,lastupdate,note
0,DB,27-MAR-17 00:00:00,4529349,5,4384,51343,51343,51476,51476,1000211,NaN,NaN,NaN,NaN,NaN,NaN,09-APR-17 09:00:06,NaN


In [6]:
# Drop unnecessary columns in leavetimes

leavetimes = leavetimes.drop(['datasource', 'vehicleid', 'passengers', 'passengersin', 'passengersout', 'distance', 'suppressed', 'justificationid', 'note'], axis=1)

In [7]:
trips.head(1)

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
0,DB,09-FEB-16 00:00:00,2824642,145,145_105,2,45668,40800,45375.0,40782.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2426901,"


In [8]:
# Get tripids as list

trip_ids = trips['tripid'].tolist()

In [9]:
leavetimes.head(3)

,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,lastupdate
0,27-MAR-17 00:00:00,4529349,5,4384,51343,51343,51476,51476,09-APR-17 09:00:06
1,27-MAR-17 00:00:00,4529349,10,523,51544,51544,51744,51763,09-APR-17 09:00:06
2,27-MAR-17 00:00:00,4529349,15,4382,51764,51764,52048,52077,09-APR-17 09:00:06


In [10]:
# Search for leavetimes for any of the value in trip_ids

leavetimes_id = leavetimes['tripid'].tolist()

In [11]:
# Find tripids shared by both

shared_tripids = [tripid for tripid in trip_ids if tripid in leavetimes_id]

In [12]:
# Create a mask True/False of which tripsids are in both dataframes
# trips.loc[trips['tripid'] == 4093258]

mask = trips['tripid'].isin(shared_tripids)

In [13]:
trips.loc[mask].head(20)

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
864,DB,03-JAN-17 00:00:00,4093258,15B,15B_56,1,25839,23820,25629.0,23861.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2754318,"
892,DB,03-JAN-17 00:00:00,4093357,15B,15B_55,1,84603,81900,84049.0,81660.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2411753,"
893,DB,03-JAN-17 00:00:00,4093335,15B,15B_55,1,81003,78300,81725.0,79197.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2762529,2762529,"
894,DB,03-JAN-17 00:00:00,4093354,15B,15B_55,1,82803,80100,82419.0,80127.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2749938,"
895,DB,03-JAN-17 00:00:00,4093329,15B,15B_55,1,86403,83700,86181.0,83657.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2764373,"
2371,DB,03-JAN-17 00:00:00,4084281,66,66_13,2,29802,25500,29379.0,25481.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2296392,"
2372,DB,03-JAN-17 00:00:00,4084355,66,66_13,2,28002,23700,26746.0,23681.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2284059,"
2373,DB,03-JAN-17 00:00:00,4084279,66,66_13,2,31602,27300,30816.0,27310.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2294414,"
2374,DB,03-JAN-17 00:00:00,4084337,66,66_13,2,26202,21900,25182.0,21916.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",2289510,"
5023,DB,03-JAN-17 00:00:00,4084412,66,66_11,1,80060,76500,80352.0,76508.0,BasDef,NaN,NaN,NaN,20-JAN-17 16:24:19,",1744594,"


As can be seen, the 15B had a trip with id 4093258

In [14]:
# Find leavetimes associated with this id

trip_leavetimes = leavetimes.loc[leavetimes['tripid'] == 4093258]
trip_leavetimes

,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,lastupdate
902,17-JAN-17 00:00:00,4093258,25,2917,24873,24873,24941,24941,16-FEB-17 09:48:02
907,17-JAN-17 00:00:00,4093258,26,2918,24904,24904,24960,24960,16-FEB-17 09:48:02
912,17-JAN-17 00:00:00,4093258,27,1300,24949,24949,25045,25045,16-FEB-17 09:48:02


In [15]:
# Find time of planned arrivales. Note 3rd Jan 2017 was a Tuesday.

plannedtime_arr = trip_leavetimes['plannedtime_arr'].tolist()
print(plannedtime_arr)

planned_arrival_times = []
for i in plannedtime_arr:
    planned_arrival_times.append(str(datetime.timedelta(seconds=i)))
    
print(planned_arrival_times)

[24873, 24904, 24949]
['6:54:33', '6:55:04', '6:55:49']


In [16]:
# Timetable information can be found here:
# https://data.smartdublin.ie/cgi-bin/rtpi/timetableinformation?type=week&stopid=2917&routeid=15B

stopid = "2917"
lineid = "15B"

timetable_request = requests.get("https://data.smartdublin.ie/cgi-bin/rtpi/timetableinformation?type=week&stopid={0}&routeid={1}".format(stopid, lineid))

In [17]:
timetableJson = json.loads(timetable_request.text)

In [18]:
timetable = json_normalize(timetableJson['results'])
timetable = timetable[['lastupdated', 'startdayofweek', 'enddayofweek', 'destination', 'destinationlocalized', 'departures']]
timetable

,lastupdated,startdayofweek,enddayofweek,destination,destinationlocalized,departures
0,27/06/2016 08:51:05,Monday,Friday,Whitechurch Green,Faiche an Teampaill,"[15:24:00, 15:32:00, 17:36:00]"
1,27/06/2016 08:51:03,Monday,Sunday,Stocking Avenue,Ascaill Stoicín,"[09:33:00, 09:35:00, 09:40:00, 10:01:00, 10:04..."
2,25/06/2018 09:43:28,Monday,Friday,Stocking Avenue,Ascaill Stoicín,"[06:38:00, 06:40:00, 06:55:00, 06:57:00, 07:09..."
3,25/06/2018 09:43:29,Saturday,Saturday,Stocking Avenue,Ascaill Stoicín,"[07:17:00, 07:24:00, 07:24:00, 07:47:00, 07:47..."


In [20]:
# Take the complete Mon-Fri schedual and print the first 10 values our times captured above are in the early morning.

monfri_timetable = timetable.iloc[2]['departures']
print(monfri_timetable[:10])
print("Leavetime planned arrival times: ", planned_arrival_times)

['06:38:00', '06:40:00', '06:55:00', '06:57:00', '07:09:00', '07:16:00', '07:20:00', '07:24:00', '07:30:00', '07:34:00']
Leavetime planned arrival times:  ['6:54:33', '6:55:04', '6:55:49']


As can be seen, the values fall somewhat in line, but do not fit perfectly...